In [1]:
pip install mysql-connector-python pandas sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import mysql.connector

conn = mysql.connector.connect(
    host="ads507-final-project.mysql.database.azure.com",
    user="minguito",
    password="ProjectGroup5",
    database="air_quality_analysis",
    ssl_ca="DigiCertGlobalRootCA.crt.pem"
)

cursor = conn.cursor()
cursor.execute("SELECT NOW();")
print(cursor.fetchall())

[(datetime.datetime(2025, 2, 9, 18, 47, 14),)]


In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Query to check if tables exist
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

# Print table names
if tables:
    print("Tables in air_quality_analysis:")
    for table in tables:
        print(table[0])
else:
    print("No tables found in the database.")

# Query to get table data - check contains data
query = "SELECT * FROM ad_viz_plotval_data_4 LIMIT 3;"

# Load data into Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Display DataFrame
print(df)

C:\Users\16302\AppData\Local\Temp\ipykernel_23904\10951791.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Tables in air_quality_analysis:
ad_viz_plotval_data
ad_viz_plotval_data_10
ad_viz_plotval_data_11
ad_viz_plotval_data_2
ad_viz_plotval_data_4
ad_viz_plotval_data_6
ad_viz_plotval_data_8
ad_viz_plotval_data_9
asthma_ed_visits
asthma_hospital_visits
pollution_data
         Date Source    Site_ID  POC  Daily_Mean_PM25_Concentration     Units  \
0  2015-01-03    AQS  060730001    1                           14.9  ug/m3 LC   
1  2015-01-06    AQS  060730001    1                            3.7  ug/m3 LC   
2  2015-01-09    AQS  060730001    1                           11.6  ug/m3 LC   

   Daily_AQI_Value Local_Site_Name  Daily_Obs_Count  Percent_Complete  ...  \
0               62     Chula Vista                1               100  ...   
1               21     Chula Vista                1               100  ...   
2               56     Chula Vista                1               100  ...   

  Method_Code                                 Method_Description CBSA_Code  \
0         145  R & P 

C:\Users\16302\AppData\Local\Temp\ipykernel_23904\10951791.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [5]:
create_table_query = """
CREATE TABLE IF NOT EXISTS pollution_data (
    Date DATE,
    Source VARCHAR(50),
    Site_ID VARCHAR(20),
    POC INT,
    Daily_Mean_PM25_Concentration FLOAT,
    Daily_Max_8hr_CO_Concentration FLOAT,
    Units VARCHAR(50),
    Daily_AQI_Value INT,
    Local_Site_Name VARCHAR(100),
    Daily_Obs_Count INT,
    Percent_Complete INT,
    AQS_Parameter_Code VARCHAR(10),
    AQS_Parameter_Description VARCHAR(100),
    Method_Code VARCHAR(20),
    Method_Description VARCHAR(255),
    CBSA_Code VARCHAR(10),
    CBSA_Name VARCHAR(100),
    State_FIPS_Code VARCHAR(10),
    State VARCHAR(50),
    County_FIPS_Code VARCHAR(10),
    County VARCHAR(50),
    Site_Latitude FLOAT,
    Site_Longitude FLOAT
);

"""

cursor.execute(create_table_query)
conn.commit()
print("Table pollution_data created successfully.")

Table pollution_data created successfully.


In [6]:
insert_query = """
INSERT INTO pollution_data (
    Date, Source, Site_ID, POC,
    Daily_Mean_PM25_Concentration, Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
)
SELECT 
    Date, Source, Site_ID, POC,
    Daily_Mean_PM25_Concentration, NULL AS Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    NULL AS Daily_Mean_PM25_Concentration, Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, NULL AS Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_2

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    Daily_Mean_PM25_Concentration, NULL AS Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_4

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    NULL AS Daily_Mean_PM25_Concentration, Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, NULL AS Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_10

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    Daily_Mean_PM25_Concentration, NULL AS Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_11

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    NULL AS Daily_Mean_PM25_Concentration, Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, NULL AS Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_6

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    Daily_Mean_PM25_Concentration, NULL AS Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_8

UNION ALL

SELECT 
    Date, Source, Site_ID, POC,
    NULL AS Daily_Mean_PM25_Concentration, Daily_Max_8hr_CO_Concentration,
    Units, Daily_AQI_Value, Local_Site_Name,
    Daily_Obs_Count, Percent_Complete, AQS_Parameter_Code,
    AQS_Parameter_Description, Method_Code, NULL AS Method_Description,
    CBSA_Code, CBSA_Name, State_FIPS_Code, State,
    County_FIPS_Code, County, Site_Latitude, Site_Longitude
FROM ad_viz_plotval_data_9;

"""

cursor.execute(insert_query)
conn.commit()
print("Data successfully inserted into pollution_data.")

ProgrammingError: 1054 (42S22): Unknown column 'Source' in 'field list'

In [ ]:
conn.close()